In [1]:
from detr_config import Config
from detr_model_trainer import ModelTrainer
from detr_model_evaluator import ModelEvaluator
from detr_model_loader import ModelLoader
from detr_file_manager import FileManager
from detr_metrics import MetricsAggregator
from detr_model_loader import ModelLoader
from detr_factory import DETRFactory
from detr_data_source import DataSource

In [2]:
# Config.set_local_settings()
Config.set_gpu_settings()

for hyperparams in Config.HYPERPARAMS:
    architecture, num_queries, transformer_layers = hyperparams
    detr_factory = DETRFactory(architecture, num_queries, transformer_layers)    
    file_manager = FileManager(detr_factory)
    data_source = DataSource(detr_factory, file_manager)
    model_loader = ModelLoader(detr_factory, file_manager)
    model_trainer = ModelTrainer(file_manager)
    metrics_aggregator = MetricsAggregator(file_manager)
    
    file_manager.clean_model_logs()

    # -----------------------
    # K-fold Cross Validation
    # -----------------------

    for fold in Config.FOLDS:

        file_manager.set_validation_setup(fold)

        # Model
        
        model = model_loader.new_pretrained_model()
        
        # Dataset
                
        train_dataset, train_loader = data_source.get_train_dataset_dataloader()
        valid_dataset, valid_loader = data_source.get_valid_dataset_dataloader()

        # Training

        model_trainer.fit(model, train_loader, valid_loader)
        
        # Validation - Threshold Optimization
        
        best_model = model_loader.load_best_model()
        model_evaluator = ModelEvaluator(best_model, detr_factory)

        for threshold in Config.THRESHOLDS:
            valid_metrics = model_evaluator.evaluate(valid_dataset, valid_loader, threshold)
            metrics_aggregator.add_metrics(threshold, valid_metrics)
        
        file_manager.clean_checkpoints()        
        
    
    metrics_aggregator.finish_validation()
    metrics_aggregator.save_metrics()
    
    # -----------------------
    # Testing
    # -----------------------
    
    # Model
    
    model = model_loader.new_pretrained_model()

    # Dataset
    
    file_manager.set_testing_setup()

    train_valid_dataset, train_valid_loader = data_source.get_train_valid_dataset_dataloader()
    test_dataset, test_loader = data_source.get_test_dataset_dataloader()
    
    # Training
    
    model_trainer.fit(model, train_valid_loader, test_loader)
    

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Cleaning Logs:  lightning_logs\model=DETR_queries=100_layers=6
New Model:  model=DETR_queries=100_layers=6


Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Loading Annotations from:  InBreast_Coco\fold_1\train\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading Annotations from:  InBreast_Coco\fold_1\valid\_annotations.coco.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!



  | Name | Type                   | Params
------------------------------------------------
0 | detr | DetrForObjectDetection | 41.5 M
------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.007   Total estimated model params size (MB)


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]